# Using Word2Vec Algorithm

It works on unlabelled data

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import spacy
import gensim
import cython

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
lab_df = pd.read_csv('drive/My Drive/Pytorch_DataSet/Bag Of Words Meets Bags of popcorn/labeledTrainData.tsv',sep='\t')
unlab_df = pd.read_csv('drive/My Drive/Pytorch_DataSet/Bag Of Words Meets Bags of popcorn/unlabeledTrainData.tsv',delimiter="\t", quoting=3)
test_df = pd.read_csv('drive/My Drive/Pytorch_DataSet/Bag Of Words Meets Bags of popcorn/testData.tsv',sep='\t')

In [5]:
lab_df.head(2)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."


In [6]:
unlab_df.head(2)

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."


In [7]:
len(lab_df), len(unlab_df), len(test_df)

(25000, 50000, 25000)

In [8]:
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [0]:
# Cleaning text
"""
def clean_text(text):
  text = text.lower()
  text = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});','',text)  # for removal of html tags
  text = re.sub(r"[^a-zA-Z]"," ",text)
  text = re.sub('\W', ' ', text)  # If the comment/word does not contain any alphabets
  text = text.strip(' ') # Removing leading and trailing white spaces
  text = text.split()  # For words splitting
  return text
"""
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

def review_to_wordlist(review, remove_stopwords=False ):
    
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    #review_text = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});','',review)  

    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)  

In [10]:
"""
lab_df['review'] = lab_df['review'].apply(lambda text: clean_text(text))
unlab_df['review'] = unlab_df['review'].apply(lambda text: clean_text(text))
test_df['review'] = test_df['review'].apply(lambda text: clean_text(text))
"""

"\nlab_df['review'] = lab_df['review'].apply(lambda text: clean_text(text))\nunlab_df['review'] = unlab_df['review'].apply(lambda text: clean_text(text))\ntest_df['review'] = test_df['review'].apply(lambda text: clean_text(text))\n"

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [24]:
import nltk.data
nltk.download('punkt')   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
tokenizer

In [0]:
# Define a function to split a review into parsed sentences

def review_to_sentences( review, tokenizer, remove_stopwords=False ):

    # Function to split a review into parsed sentences. Returns a list of sentences, where each sentence is a list of words
    
    # 1. Use the NLTK tokenizer to split the paragraph into sentences

    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence

    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist( raw_sentence, remove_stopwords ))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [31]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in lab_df["review"]:
    #print(type(review))
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlab_df["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set
Parsing sentences from unlabeled set


In [32]:
print(len(sentences))

795872


In [34]:
' '.join(sentences[0])

'with all this stuff going down at the moment with mj i ve started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again'